## Cohere's Aya-23-8B
Aya-23-8B is the latest open source models from Cohere which is derived from the Cohere's bigger and closed source Command Family. This smaller open sourced family of models come in 2 sizes, the 8 billion parameter model and the 35 billion parameters and supports upto 23 languages.
You can find the complete details of the models in the [model card](https://huggingface.co/CohereForAI/aya-23-8B).

In this notebook I will be downloading the 8B model and test it's translation abilities.

###Install the required Dependencies

In [1]:
!pip install transformers==4.41.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 73.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [2]:
!pip install huggingface-hub

In [3]:
!pip install pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 91.8 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [5]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 9.4 MB/s eta 0:00:00


In [6]:
from huggingface_hub import notebook_login                    # Log into your hugging face account using the secret token
                                                              # Copy paste the Hugging-Face access keys in the cell below
notebook_login()

###Initialise the quantisation parameters and load the quantised model.

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [8]:
model_id = "CohereForAI/aya-23-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="auto",
    )


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/17.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

###Define a function to translate the text.

In [9]:
def translate(text, lang):
    messages = [{"role": "user",
                 "content": f''' Translate the following text: {text}
                                 from english to this language '{lang}' '''}]
    input_ids = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=False, return_tensors="pt")


    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=2000,
        do_sample=True,
        temperature=0.3,
        )

    gen_text = tokenizer.decode(gen_tokens[0])
    # Extract the translated text between the special tokens
    start_token = "<EOS_TOKEN>"
    end_token = "<EOS_TOKEN>"

    start_idx = gen_text.find(start_token) + len(start_token)
    end_idx = gen_text.find(end_token, start_idx)

    translated_text = gen_text[start_idx:end_idx].strip()

    return translated_text


###Testing the model's translation capabilities by translating a sample paragraph from English to all the supported languages.

In [11]:
languages = [
    "Arabic",
    "Chinese (simplified)",
    "Chinese (traditional)",
    "Czech",
    "Dutch",
    "French",
    "German",
    "Greek",
    "Hebrew",
    "Hindi",
    "Indonesian",
    "Italian",
    "Japanese",
    "Korean",
    "Persian",
    "Polish",
    "Portuguese",
    "Romanian",
    "Russian",
    "Spanish",
    "Turkish",
    "Ukrainian",
    "Vietnamese"
]

Test_Sentence = '''Vegetables are an essential component of a healthy diet, providing a rich source of vitamins,
minerals, fiber, and antioxidants. They come in a wide variety of types and colors, each offering unique
health benefits. Leafy greens like spinach and kale are packed with iron and calcium, while root vegetables
like carrots and sweet potatoes are high in beta-carotene and vitamin A. Cruciferous vegetables such as
broccoli and Brussels sprouts contain compounds that may have cancer-preventive properties.
Incorporating a diverse array of vegetables into daily meals can support overall health,
boost immune function, and reduce the risk of chronic diseases.'''

outdict={}


for language in languages:
    translated_text = translate(Test_Sentence, language)
    outdict[language] = translated_text


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [12]:
outdict["Arabic"]

'في إدراج مجموعة متنوعة من الخضروات في الوجبات اليومية، يمكن دعم الصحة العامة، وتعزيز وظيفة المناعة، وتقليل خطر الأمراض المزمنة.<|END_OF_TURN_TOKEN|'

In [13]:
outdict["Chinese (simplified)"]

'我 认为 健康 饮食 的 重要 组成部分 是 蔬菜， 它们 能 提供 丰富的 维生素、 矿物质、 纤维 和 抗氧化 剂。 它们 有 多种 类型 和 颜色， 每 种 都有 独特 的 健康 益处。 如 叶 菜 类 如 菠 菜 和 叶 菜， 根 菜 类 如 胡 萝 卜 和 甜 薯， 以及 十字 花 菜 类 如  broccoli 和  Brussels 芽。 这些 蔬菜 含有 可 预防 癌症 的 化合物。 每天 摄入 多 种 蔬菜 可 支持 整体 健康、 提升 免疫 功能、 减少 慢性 疾病 的 风险。<|END_OF_TURN_TOKEN|'

In [14]:
outdict["Chinese (traditional)"]

'我 們 吃 的 蔬菜 是 健康 饮食 的 重要 部分， 能 提供 充 足 的 維 生素、 微 量 元素、 纖 維 和 抗 氧化 劑。 它 們 有 多 樣 的 種類 和 色 彩， 每 一 種 都 能 帶 來 獨 特 的 健 康 益 益。 如 果 葉 綠 菜 類， 如 菠 菜 和 葉 菜， 就 富 含 鐵 和 钙。 根 菜 類， 如 胡 萝 卜 和 甜 土 豆， 則 含 有 高 量 的 β- 碳 素 和 維 生素 A。 如 果 結 節 菜 類， 如  broccoli 和  Brussels 芽， 就 含 有 可 能 有 防 癌 功 效 的 合 質。 每 天 吃 多 樣 的 菜 色 能 支 持 整 體 的 健 康、 提 高 免疫 能 力， 並 減 低 慢性 病 的 機 率。<|END_OF_TURN_TOKEN|'

In [15]:
outdict["Czech"]

'"Zelenina je nezbytnou složkou zdravé stravy, která poskytuje bohaté zdroje vitamínů, minerálů, vlákniny a antioxidantů. Je k dispozici v široké škále typů a barev, každý z nich nabízí jedinečné zdravotní výhody. Listová zelenina, jako je špenát a kale, je bohatá na železo a vápník, zatímco kořenová zelenina, jako jsou mrkve a sladké brambory, obsahuje vysoké množství beta-karotenu a vitamínu A. Zelená zelenina, jako je brokolice a brušinky, obsahuje sloučeniny, které mohou mít preventivní účinky rakoviny."<|END_OF_TURN_TOKEN|'

In [16]:
outdict["Dutch"]

'Het eten van groenten is een essentieel onderdeel van een gezonde voeding, en biedt een rijke bron van vitaminen, mineralen, vezels en antioxidanten. Ze komen in een breed scala aan soorten en kleuren, en bieden unieke gezondheidsvoordelen voor elk van hen. Bladgroenten zoals spinazie en kale zijn rijk aan ijzer en calcium, terwijl wortelgroenten zoals wortelen en zoete aardappelen hoog zijn in bèta-caroteen en vitamine A. Kruisbloemige groenten zoals broccoli en spruitjes bevatten verbindingen die mogelijk kankerpreventieve eigenschappen hebben. Door een divers scala aan groenten in dagelijkse maaltijden op te nemen, kan dit de algehele gezondheid ondersteunen, de immuunfunctie verbeteren en het risico op chronische ziekten verminderen.<|END_OF_TURN_TOKEN|'

In [17]:
outdict["French"]

"Voici la traduction :\n\nLes légumes sont un élément essentiel d'un régime alimentaire sain, offrant une source abondante de vitamines, de minéraux, de fibres et d'antioxydants. Ils présentent une grande variété de types et de couleurs, chacun offrant des bienfaits uniques pour la santé. Les légumes feuillus comme le spinat et le chou frisé sont riches en fer et en calcium, tandis que les légumes-racines comme les carottes et les patates douces sont chargés de bêta-carotène et de vitamine A. Les légumes crucifères tels que le brocoli et les choux de Bruxelles contiennent des composés qui pourraient avoir des propriétés anticancéreuses. L'incorporation d'une grande variété de légumes dans les repas quotidiens peut soutenir la santé globale, renforcer les fonctions immunitaires et réduire le risque de maladies chroniques.<|END_OF_TURN_TOKEN|"

In [18]:
outdict["German"]

'Die Gemüse sind ein wesentlicher Bestandteil einer gesunden Ernährung, sie bieten eine reiche Quelle an Vitaminen, Mineralstoffen, Ballaststoffen und Antioxidantien. Sie gibt es in einer großen Vielfalt an Typen und Farben, jede mit einzigartigen gesundheitlichen Vorteilen. Blattgemüse wie Spinat und Kale sind reich an Eisen und Kalzium, während Wurzelgemüse wie Karotten und Süßkartoffeln viel Beta-Karotin und Vitamin A enthalten. Kreuzblütlergemüse wie Brokkoli und Brüsseler Sprossen enthalten Verbindungen, die möglicherweise krebshemmende Eigenschaften besitzen. Durch die Aufnahme einer vielfältigen Auswahl an Gemüse in den täglichen Speiseplan kann die allgemeine Gesundheit, die Immunfunktion und die Reduzierung des Risikos chronischer Krankheiten unterstützt werden.<|END_OF_TURN_TOKEN|'

In [19]:
outdict["Greek"]

'Η κατανάλωση λαχανικών αποτελεί βασικό συστατικό μιας υγιεινής διατροφής, παρέχοντας πλούσια πηγή βιταμινών, μεταλλικών στοιχείων, ινών και αντιοξειδωτικών. Έχουν μια μεγάλη ποικιλία τύπων και χρωμάτων, καθένας από τους οποίους προσφέρει μοναδικά οφέλη για την υγεία. Τα πράσινα φυλλώδη λαχανικά όπως το σπανάκι και το kale είναι γεμάτα με σίδηρο και ασβέστιο, ενώ τα λαχανικά ρίζας όπως τα καρότα και οι γλυκοπατάτες είναι πλούσια σε β-καροτένιο και βιταμίνη Α. Τα λαχανικά κρουσολέους όπως το μπρόκολο και τα σπαράγγια περιέχουν ενώσεις που μπορεί να έχουν προληπτικές ιδιότητες κατά του καρκίνου. Η ενσωμάτωση μιας ποικιλίας λαχανικών στις καθημερινές γεύσεις μπορεί να υποστηρίξει την συνολική υγεία, να ενισχύσει τη λειτουργία του ανοσοποιητικού συστήματος και να μειώσει τον κίνδυνο χρόνιων παθήσεων.<|END_OF_TURN_TOKEN|'

In [20]:
outdict["Hebrew"]

'הפירות והירקות הם מרכיב חיוני בתזונה בריאה, המספקת מקור עשיר של ויטמינים, מינרלים, סיבים ואנטי-אוקסידנטים. יש מגוון רחב של סוגים וצבעים, וכל אחד מהם מציע יתרונות בריאותיים ייחודיים. עלים ירוקים כמו תרד וחסה עמוסים בברזל ובסידן, בעוד שורש ירקות כמו גזרים ותפוחי אדמה עשירים בבטא-קרוטן ובחומצה ויטאמין A. ירקות צולבניים כמו ברוקולי וחסה ברוסלים מכילים חומרים אשר עשויים להיות בעלי תכונות מניעת סרטן. שילוב מגוון רחב של ירקות בארוחות יומיות יכול לתמוך בבריאות הכללית, לחזק את תפקוד המערכת החיסונית ולהפחית את הסיכון למחלות כרוניות.<|END_OF_TURN_TOKEN|'

In [21]:
outdict["Hindi"]

'यहाँ एक अंग्रेज़ी का पार्ग है जिसे हिंदी में अनुवादित किया गया है:\n\nस्वस्थ आहार के लिए सब्जियाँ एक महत्वपूर्ण घटक हैं, जो विटामिन, खनिज, फाइबर और एंटीऑक्सिडेंट का एक समृद्ध स्रोत प्रदान करती हैं। वे विभिन्न प्रकार और रंगों में आते हैं, प्रत्येक में अद्वितीय स्वास्थ्य लाभ होते हैं। स्पिनाच और केल जैसी पत्तेदार सब्जियाँ लोहे और कैल्शियम से भरपूर होती हैं, जबकि गाजर और मीठे आलू जैसी जड़ सब्जियाँ बीटा-कारोटीन और विटामिन ए से उच्च होती हैं। ब्रोकोली और ब्रुस्सेल्स स्प्राउट्स जैसी क्रूसिफ़रस सब्जियाँ कैंसर-रोधी गुणों वाले कंपाउंड्स को प्रदान करती हैं। रोजमर्रा के भोजन में विभिन्न प्रकार की सब्जियों को शामिल करने से सामान्य स्वास्थ्य, प्रतिरक्षा कार्य को बढ़ावा मिलता है और चोटिल बीमारियों के जोखिम को कम किया जा सकता है।<|END_OF_TURN_TOKEN|'

In [22]:
outdict["Indonesian"]

'Terjemahan dari teks tersebut dalam bahasa Inggris ke bahasa Indonesia:\n\nSayuran merupakan komponen penting dalam diet sehat, yang menyediakan sumber kaya akan vitamin, mineral, serat, dan antioksidan. Mereka memiliki berbagai jenis dan warna, masing-masing menawarkan manfaat kesehatan yang unik. Sayuran berdaun hijau seperti bayam dan kale kaya akan zat besi dan kalsium, sementara sayuran akar seperti wortel dan kentang manis memiliki kandungan beta-karoten dan vitamin A yang tinggi. Sayuran silikoni seperti brokoli dan kubis Brussel mengandung senyawa yang mungkin memiliki sifat pencegahan kanker.\n\nMenyertakan berbagai macam sayuran dalam makanan harian dapat mendukung kesehatan secara keseluruhan, meningkatkan fungsi kekebalan tubuh, dan mengurangi risiko penyakit kronis.<|END_OF_TURN_TOKEN|'

In [23]:
outdict["Italian"]

"La verdura è un componente essenziale di una dieta sana, fornendo una fonte ricca di vitamine, minerali, fibre e antiossidanti. Esistono in una vasta varietà di tipi e colori, ciascuno con benefici per la salute unici. Le verdure a foglia come la spinacia e il cavolo sono ricche di ferro e calcio, mentre le verdure a radice come le carote e le patate dolci sono alte in beta-carotene e vitamina A. Le verdure crucifere come il broccoli e gli spinaci di Bruxelles contengono composti che potrebbero avere proprietà preventive del cancro. L'inclusione di una vasta gamma di verdure nei pasti quotidiani può sostenere la salute generale, potenziare la funzione immunitaria e ridurre il rischio di malattie croniche.<|END_OF_TURN_TOKEN|"

In [24]:
outdict["Japanese"]

'"野菜は健康的な食生活に欠かせない要素であり、ビタミン、ミネラル、食物繊維、抗酸化物質を豊富に含んでいます。色や種類が豊富で、それぞれに異なる健康効果があります。ほうれん草やケールなどの葉物野菜は鉄分やカルシウムを豊富に含み、ニンジンやサツマイモなどの根菜類はベータカロテンやビタミンAを多く含みます。ブロッコリーやブリュッセルスプラウトなどのアブラナ科野菜には、癌予防効果がある可能性がある化合物が含まれています。毎日様々な野菜を食事に取り入れることで、健康をサポートし、免疫機能を高め、慢性疾患のリスクを軽減することができます。"<|END_OF_TURN_TOKEN|'

In [25]:
outdict["Korean"]

'번역: 채소는 건강한 식단의 필수 요소입니다. 비타민, 미네랄, 섬유질, 항산화제가 풍부합니다. 다양한 종류와 색깔의 채소가 있으며, 각기 독특한 건강 효능을 제공합니다. 시금치와 케일 같은 잎채소는 철분과 칼슘이 풍부하며, 당근과 감자 같은 뿌리채소는 베타카로틴과 비타민 A가 높습니다. 브로콜리와 브뤼셀 콩나물 같은 십자화과 채소는 암 예방 성분을 함유하고 있습니다. 다양한 종류의 채소를 매일 식사에 포함시키는 것은 전반적인 건강을 지원하고, 면역 기능을 강화하며, 만성 질환의 위험을 줄이는 데 도움이 됩니다.<|END_OF_TURN_TOKEN|'

In [26]:
outdict["Persian"]

'ترجمه از انگلیسی به فارسی:\nسبزیجات یکی از اجزای ضروری رژیم غذایی سالم هستند، که منبع غنی از ویتامین ها، مواد معدنی، فیبر و آنتی اکسیدان ها می باشند. انواع و اقلام مختلفی دارند که هر کدام فواید سلامتی منحصر به فردی دارند. سبزیجات برگ دار مانند اسفناج و کلم بروکلی سرشار از آهن و کلسیم هستند، در حالی که سبزیجات ریشه ای مانند هویج و سیب زمینی شیرین حاوی بتاکاروتن و ویتامین A هستند. سبزیجات کلمی مانند بروکلی و کلم بروکسل حاوی ترکیباتی هستند که ممکن است خواص پیشگیرانه از سرطان داشته باشند.\nگنجاندن طیف متنوعی از سبزیجات در وعده های غذایی روزانه می تواند از سلامت کلی، تقویت عملکرد سیستم ایمنی و کاهش خطر بیماری های مزمن پشتیبانی کند.<|END_OF_TURN_TOKEN|'

In [27]:
outdict["Polish"]

'O warzywach jako składniku niezbędnym zdrowej diety, które dostarczają bogactwa witamin,\nminerałów, błonnika i przeciwutleniaczy. Są one dostępne w szerokiej gamie rodzajów i kolorów, każdy oferując unikalne\nwłaściwości zdrowotne. Liście zielone, takie jak szpinak i kale, są bogate w żelazo i wapń, podczas gdy warzywa korzeniowe,\ntakie jak marchew i bataty, są bogate w beta-karoten i witaminę A. Warzywa strączkowe, takie jak brokuły i brukselka, zawierają związki,\nktóre mogą mieć właściwości zapobiegające rakowi. Włączenie różnorodnej gamy warzyw do codziennych posiłków może wspierać ogólne\nzdrowie, poprawiać funkcję odpornościową i zmniejszać ryzyko chorób przewlekłych.<|END_OF_TURN_TOKEN|'

In [28]:
outdict["Portuguese"]

'A vegetais são um componente essencial de uma dieta saudável, fornecendo uma rica fonte de vitaminas, minerais, fibras e antioxidantes. Eles vêm em uma ampla variedade de tipos e cores, cada um oferecendo benefícios de saúde exclusivos. As folhas verdes como a espinafre e a couve são ricas em ferro e cálcio, enquanto os vegetais de raiz como os cenouras e batatas doces são altas em beta-caroteno e vitamina A. Os vegetais crucíferos, como o brócolis e os brotos de brócolis, contêm compostos que podem ter propriedades preventivas de câncer. A inclusão de uma ampla variedade de vegetais em refeições diárias pode apoiar a saúde geral, melhorar a função imunológica e reduzir o risco de doenças crônicas.<|END_OF_TURN_TOKEN|'

In [29]:
outdict["Romanian"]

'Traducere:\nLegumele reprezintă un component esențial al unei diete sănătoase, oferind o sursă bogată de vitamine, minerale, fibre și antioxidanți. Acestea se prezintă într-o varietate largă de tipuri și culori, fiecare având beneficii unice pentru sănătate. Legumele cu frunze, cum ar fi spanacul și kale, sunt bogate în fier și calciu, în timp ce legumele rădăcinoase, cum ar fi morcovii și cartofii dulci, conțin beta-caroten și vitamina A. Legumele crucifere, cum ar fi broccoli și varza de Bruxelles, conțin compuși care pot avea proprietăți anticancerigene.\nIntegrarea unei game variate de legume în mesele zilnice poate sprijini sănătatea generală, îmbunătăți funcția imunitară și reduce riscul de boli cronice.<|END_OF_TURN_TOKEN|'

In [30]:
outdict["Russian"]

'Здоровое питание включает в себя овощи, которые являются важной составляющей здоровой диеты, обеспечивая богатый источник витаминов, минералов, клетчатки и антиоксидантов. Они бывают разных типов и цветов, и каждый из них обладает уникальными полезными свойствами. Листовые овощи, такие как шпинат и капуста, богаты железом и кальцием, в то время как корневые овощи, такие как морковь и сладкий картофель, содержат много бета-каротина и витамина А. Кружевые овощи, такие как брокколи и брюссельская капуста, содержат соединения, которые могут иметь противораковые свойства. Включение в ежедневное питание разнообразных овощей может способствовать поддержанию общего здоровья, укреплению иммунной системы и снижению риска хронических заболеваний.<|END_OF_TURN_TOKEN|'

In [31]:
outdict["Spanish"]

'La verdura es un componente esencial de una dieta saludable, proporcionando una fuente rica de vitaminas, minerales, fibra y antioxidantes. Viene en una amplia variedad de tipos y colores, cada uno ofreciendo beneficios únicos para la salud. Las verduras de hoja como la espinaca y la col rizada están llenas de hierro y calcio, mientras que las verduras de raíz como las zanahorias y las batatas dulces son altas en beta-caroteno y vitamina A. Las verduras crucíferas como el brócoli y las coles de Bruselas contienen compuestos que pueden tener propiedades preventivas del cáncer. Incorporar una amplia variedad de verduras en las comidas diarias puede apoyar la salud en general, mejorar la función inmune y reducir el riesgo de enfermedades crónicas.<|END_OF_TURN_TOKEN|'

In [32]:
outdict["Turkish"]

'"Sebze, sağlıklı bir beslenmenin temel bileşenidir ve vitamin, mineral, lif ve antioksidan bakımından zengindir. Çeşitli türleri ve renkleri vardır ve her biri benzersiz sağlık faydaları sunar. Yapraklı sebzeler gibi ıspanak ve kale, demir ve kalsiyum bakımından zengindir, havuç ve tatlı patates gibi kök sebzeleri ise beta-karoten ve A vitamini bakımından yüksektir. Brokoli ve Brüksel lahanası gibi çapraz yapraklı sebzeler kansere karşı koruyucu özelliklere sahip olabilecek bileşikler içerir. Her gün yemeklere çeşitli sebzeler eklemek genel sağlığı destekleyebilir, bağışıklık fonksiyonunu artırabilir ve kronik hastalık riskini azaltabilir."<|END_OF_TURN_TOKEN|'

In [33]:
outdict["Ukrainian"]

"Овочі є невід'ємною складовою здорового раціону, забезпечуючи багатий джерело вітамінів,\nмінеральних речовин, клітковини та антиоксидантів. Вони мають широкий спектр типів і кольорів, кожен з яких\nпропонує унікальні переваги для здоров'я. Листяні зелені, такі як шпинат і кале, багаті залізом і кальцієм,\nа коренеплоди, такі як морква і солодкий картопля, містять бета-каротин і вітамін А у високих\nконцентраціях. Цвітна капуста та брюссельська капуста, що належать до родини хрестоцвітих, містять\nкомплекси, які можуть мати протипухлинні властивості. Включення різноманітних овочів у щоденні страви\nможе сприяти загальному здоров'ю, підвищенню імунної функції та зниженню ризику хронічних захворювань.<|END_OF_TURN_TOKEN|"

In [34]:
outdict["Vietnamese"]

"The translation of the text in the language 'Vietnamese' is: Rau củ là một thành phần thiết yếu của chế độ ăn uống lành mạnh, cung cấp nguồn giàu vitamin, khoáng chất, chất xơ và chất chống oxy hóa. Chúng có nhiều loại và màu sắc khác nhau, mỗi loại mang lại lợi ích sức khỏe độc đáo. Các loại rau lá xanh như cải bó xôi và cải thìa là giàu sắt và canxi, trong khi các loại rau củ rễ như cà rốt và khoai lang giàu beta-carotene và vitamin A. Các loại rau họ cải như bông cải xanh và súp lơ chứa các hợp chất có thể có tác dụng phòng ngừa ung thư. Việc kết hợp nhiều loại rau củ khác nhau vào các bữa ăn hàng ngày có thể hỗ trợ sức khỏe tổng thể, tăng cường chức năng miễn dịch và giảm nguy cơ mắc bệnh mãn tính.<|END_OF_TURN_TOKEN|"

The model is able to translate the given text to all these languages but I can only verify the quality of the Hindi translation as that is the only language I'm familiar with. The quality of Hindi translation is pretty good and I expect similar quality for the other languages as well. This is very impressive especially for an open-source model which can be further fine tuned for specific use cases. Plus, I've only tested the model's capabilities in tranlating english to other languages, this model also supports translation between languages which is amazing!